# Sequence Models & How to use them

**학습목표**
1. Bidirectional layer를 이해한다.
2. Bidirectional layer를 사용할 줄 안다.

-----------------
* Thanks to : 한기영 대표님 @ Data Insight

## 1.환경 및 데이터 준비

### Import Packages

In [ ]:
#라이브러리들을 불러오자.




### Data Loading

In [ ]:
url = 'https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/Stock_Edwards_Lifesciences_corporation.csv'

# 판다스로 데이터를 불러오시오.
data = 

##3.데이터 준비

### Date 컬럼을 제거하시오.

## 4.Sequence 데이터 구조로 만들기

**조건**
* 내일의 Close를 예측할 것이다. 이를 Y로 둘 것.
* timestep은 10주를 본다. (주식은 5일이 1주일)
* 맞추어 전처리

**데이터 분할 규칙**
* 가장 최근 1주일을 테스트 데이터로 둔다.
* 테스트 데이터를 제외하고, 가장 최근 2주를 벨리데이션 데이터로 둔다.

In [ ]:
# 판다스 데이터 프레임을 넘파이로 옮김.
data = data.values

In [ ]:
# 여기에 의미있는 기간(timestep을 지정해 봅시다.)
timestep= 




## 모양 확인 필수



데이터의 구조 : [n, timestep, feature수]

# 같이 해보기
**[참고링크](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional)**
1. 적절한 인풋 레이어를 구성할 것
2. Conv1D
    * 16개 필터 사용할 것
    * window_size(filter_size)는 5일
    * activation은 swish
3. MaxPool1D
    * 필터 사이즈(window size)는 2일
4. Bidirectional 레이어
    * LSTM, 히든스테이트 노드 32개
5. Bidirectional 레이어
    * LSTM, 히든스테이트 노드 32개
4. 플래튼
5. 회귀를 위한 적절한 아웃풋 레이어



In [ ]:
####################
## Your Code Here ##
####################


In [ ]:
## 학습도 시킬 것



In [ ]:
# Q1. 테스트 셋에서의 RMSE를 출력하여라.



## Q1. Conv1D + Bidirectional

**[공식문서 적극 참고](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional)**
Bidirectional)**
1. 적절한 인풋 레이어를 구성할 것
2. Conv1D
    * 32개 필터 사용할 것
    * window_size(filter_size)는 10일
    * activation은 swish
3. MaxPool1D
    * 필터 사이즈(window size)는 2일
4. Bidirectional 레이어
    * forward layer : LSTM, 히든스테이트 노드 24개
    * backward layer : GRU, 히든스테이트 노드 16개
4. Bidirectional 레이어
    * forward layer : LSTM, 히든스테이트 노드 24개
    * backward layer : GRU, 히든스테이트 노드 24개
4. 플래튼
6. Fully Connected : 노드 256개, swish
5. 회귀를 위한 적절한 아웃풋 레이어


In [ ]:
####################
## Your Code Here ##
####################



In [ ]:
## 학습도 시킬 것



In [ ]:
# Q1. 테스트 셋에서의 RMSE를 출력하여라.

